# Гайд: виртуальная среда и `uv` на Vast.ai

Этот ноутбук можно использовать как шпаргалку на Vast.ai:

* как понять, какой Python вообще запущен;
* как аккуратно создать виртуальную среду (через `uv venv` или `python -m venv`);
* как установить пакеты в **нужный** интерпретатор;
* как запускать код внутри venv и как подружить это всё с ноутбуком.


## 0. Проверка окружения (Python, диск, GPU)

In [ ]:
import sys, shutil, subprocess, os, textwrap

print("Python version:", sys.version)
print("Python executable:", sys.executable)

print("\n=== which python / pip ===")
for cmd in ["which python", "which python3", "which pip", "which pip3"]:
    try:
        out = subprocess.check_output(cmd, shell=True, text=True).strip()
    except Exception as e:
        out = f"error: {e}"
    print(f"$ {cmd}\n{out}\n")

print("=== df -h (основной диск) ===")
try:
    out = subprocess.check_output("df -h /", shell=True, text=True)
    print(out)
except Exception as e:
    print("df error:", e)

print("=== nvidia-smi (если есть GPU) ===")
try:
    out = subprocess.check_output("nvidia-smi", shell=True, text=True)
    print(out)
except Exception as e:
    print("nvidia-smi error:", e)

## 1. Установка и базовая идея `uv`

`uv` — это быстрый менеджер пакетов и виртуальных сред.  
Главные команды, которые нужны на Vast.ai:

* `uv venv .venv_name` — создать виртуальную среду;
* `uv pip install --python /path/to/python ...` — установить пакеты в конкретный интерпретатор/venv;
* `uv run python script.py` — запустить команду внутри venv (без ручного `source venv/bin/activate`).

### 1.1. Установка `uv` (если ещё нет)


In [ ]:
import sys, subprocess

try:
    import uv  # type: ignore
    print("uv уже установлен")
except ImportError:
    print("Устанавливаем uv...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "uv"])
    import uv  # noqa
    print("uv установлен")

## 2. Какой Python использовать для venv на Vast.ai

На Vast.ai часто есть несколько Python-ов:

* системный `/usr/bin/python3` (часто «управляемый» дистрибутивом → не ставим туда пакеты руками);
* кастомный `/venv/main/bin/python` или что-то подобное, с которым запускается Jupyter.

Самый безопасный вариант — использовать **тот Python, который показывает `sys.executable` в этом ноутбуке** (`/venv/main/bin/python` или аналогичный).

Скопируем путь в переменную, чтобы не забывать:


In [ ]:
import sys, os
PYTHON_MAIN = sys.executable
PYTHON_MAIN

## 3. Создание виртуальной среды через `uv venv`

### Вариант A: отдельная venv для проекта (рекомендуется)

Создаём venv в текущей папке, например `.venv_project`:

> Важно: эту команду нужно выполнять **в терминале**, не обязательно из ноутбука.

```bash
uv venv .venv_project
```

`uv` сам выберет интерпретатор (обычно тот, у которого стоит `uv`).  
Если хочешь явно указать Python:

```bash
uv venv --python /venv/main/bin/python .venv_project
```

После этого структура будет такая:

```text
./.venv_project/
    bin/python
    bin/pip
    ...
```

### Вариант B: не создавать venv, а ставить в уже существующий

Иногда Vast.ai даёт уже готовый интерпретатор, напр. `/venv/main/bin/python`.  
Тогда можно **не делать venv**, а ставить пакеты прямо туда:

```bash
uv pip install --python /venv/main/bin/python catboost autogluon.tabular
```

Ниже — пример того же, но через ячейку ноутбука.


In [ ]:
import sys, subprocess

python_for_packages = PYTHON_MAIN  # можно вручную подставить путь к нужному интерпретатору

print("Будем ставить пакеты в интерпретатор:", python_for_packages)

cmd = [
    "uv", "pip", "install",
    "--python", python_for_packages,
    "catboost", "pandas"
]
print("Команда:", " ".join(cmd))
subprocess.check_call(cmd)
print("Готово")

## 4. Как работать с venv: терминал vs notebook

### 4.1. В терминале

Если ты создал `.venv_project`:

```bash
source .venv_project/bin/activate
python -m pip install catboost pandas
python my_script.py
```

Выйти из venv:

```bash
deactivate
```

### 4.2. Через `uv run` без `activate`

Можно вообще не активировать venv руками, а использовать:

```bash
uv run --python .venv_project/bin/python python my_script.py
```

или:

```bash
uv run --python .venv_project/bin/python python -m pip install catboost
```

### 4.3. В Jupyter-ноутбуке

Важно понимать: ноутбук привязан к **одному интерпретатору**, который показывает `sys.executable`.

Есть два сценария:

1. Ты ставишь всё в тот Python, который уже использует ноут (`PYTHON_MAIN`) → просто работаешь, ничего больше не надо.
2. У тебя отдельная venv (например `.venv_project`) и ты хочешь **отдельное ядро Jupyter**.

Тогда:

```bash
uv pip install --python .venv_project/bin/python ipykernel
uv run --python .venv_project/bin/python python -m ipykernel install --user --name project-kernel --display-name "Python (project)"
```

После этого в интерфейсе Jupyter появится новое ядро `"Python (project)"`, можно переключиться на него.


## 5. Мини-пайплайн под олимпиаду: проектная venv + uv

Ниже — примерный набор команд, который можно адаптировать под любой проект.


In [ ]:
project_venv = ".venv_project"
print("Проектная venv:", project_venv)

print("\n=== Пример команд (выполнять в терминале) ===")
print(
    "uv venv --python {py} {venv}\n"
    "uv pip install --python {venv}/bin/python catboost autogluon.tabular featuretools\n"
    "uv run --python {venv}/bin/python python -c 'import catboost, autogluon, featuretools; print(\"OK\")'"
    .format(py=PYTHON_MAIN, venv=project_venv)
)

## 6. Частые проблемы и как их чинить

### 6.1. `python: command not found`

Значит, в PATH нет команды `python`.  
Используй `python3` или полный путь:

```bash
/venv/main/bin/python -m pip install ...
```

Или в uv:

```bash
uv pip install --python /venv/main/bin/python catboost
```

### 6.2. `The interpreter at /usr is externally managed`

Такое бывает, если ты пытаешься ставить пакеты в системный Python `/usr/bin/python3`.  
Лучше:

* не трогать системный Python;
* использовать venv или уже существующий пользовательский Python (`/venv/main/bin/python` и т.п.);
* всегда указывать `--python` в `uv pip install`.

### 6.3. `No space left on device (os error 28)`

Частая проблема на контейнерах.

Что можно сделать:

* удалить ненужные кеши:

```bash
rm -rf ~/.cache/pip
rm -rf ~/.cache/huggingface
rm -rf ~/.cache/uv
```

* хранить артефакты (модели, данные) в каталоге, который смонтирован на большой диск (`/workspace`, `/root/workdir` и т.п.);
* следить за `df -h`, где именно занято место.

### 6.4. Пакет "установился", но `import` в ноуте не работает

Чаще всего:

* пакет поставился **в другой интерпретатор**, а ноут использует свой.
* Проверь:

```python
import sys
print(sys.executable)
```

и убедись, что `uv pip install --python ...` указывал **тот же** путь.

Если всё ещё не работает — перезапусти ядро (`Restart kernel`) и попробуй ещё раз.


---

Теперь этот ноут можно использовать как референс:

* посмотреть актуальный `sys.executable`;
* быстро нагенерить команды для `uv venv` и `uv pip`;
* понимать, куда именно ставятся пакеты и как это связано с ядром Jupyter.

При желании можно скопировать куски из этого ноутбука прямо в боевые окружения Vast.ai и адаптировать под конкретный проект (RAG, AutoML, CatBoost и т.д.).